In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine
import re
import time 

In [2]:
username = 'admin'
password = 'nX0d4tuvxL3w5hMjJRfz'
host = 'ai-engine.cluster-cla9jbugaywl.eu-west-2.rds.amazonaws.com'
database_name = 'superleague_clubs'

connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

def insert_into_db(df, table_name) : 

    engine = create_engine(connection_url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False) 
    return 'Data inserted successfully'

In [3]:
def extractID(playerURL):

    match = re.search(r'/(\d+)$', playerURL)

    if match:
        extracted_id = match.group(1)
        return extracted_id

In [8]:
club = 'ajax-amsterdam'
club_id = '610'
matches_url = f'https://www.transfermarkt.com/{club}/spielplan/verein/{club_id}/plus/1?saison_id='
squad_url = f'https://www.transfermarkt.com/{club}/startseite/verein/{club_id}?saison_id='

startyear = 1992
endyear = 2022

In [10]:
data = []
headers = ['shirt_number', 'playername', 'player_url','mainposition', 'position', 'birthdate', 'country', 'club_name', 'market_value', 'season']


for year in range(startyear, endyear+1) : 

    cluburl = squad_url + str(year)
    print(cluburl)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") 
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")

    driver = webdriver.Chrome(options=chrome_options) 
    driver.get(cluburl)

    driver.implicitly_wait(10) 
    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, "lxml")

    table = soup.select('.responsive-table > .grid-view > .items > tbody')[0]

    for row in table.find_all(True, {"class": ["odd", "even"]}):

        try : 
            rowdata = []

            mainposition = row.find('td')['title']
            shirt_number = row.find('div', class_='tm-shirt-number').text.strip()
            player_name = row.find('td', class_='posrela').find_all('tr')[0].td.img['title']
            player_url = row.find('td', class_='posrela').find('tr', class_='data-link')['data-link']
            position_element = row.find('td', class_='posrela')
            position = position_element.find_all('tr')[1].text if position_element else '-'
            birthdate_element = row.find_all('td', class_='zentriert')[1]
            birthdate = birthdate_element.text.strip().split('(')[0].strip() if birthdate_element else '-'
            country_element = row.find_all('td', class_='zentriert')[2].img
            country = country_element['title'] if country_element else '-'
            club_element = row.find_all('td', class_='zentriert')[3].a
            club_name = club_element['title'] if club_element else '-'
            market_value_element = row.find('td', class_='rechts hauptlink').a
            market_value = market_value_element.text.strip() if market_value_element else '-'

            rowdata.append(shirt_number)
            rowdata.append(player_name)
            rowdata.append(player_url)
            rowdata.append(mainposition)
            rowdata.append(position)
            rowdata.append(birthdate)
            rowdata.append(country)
            rowdata.append(club_name)
            rowdata.append(market_value)
            rowdata.append(str(year)+'-'+str(year+1))

        except : 
            continue

        data.append(rowdata)

clubdf = pd.DataFrame(data, columns=headers)   
insert_into_db(clubdf, f'transfermarkt_players_{club}') 

https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=1998
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=1999
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2000
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2001
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2002
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2003
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2004
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2005
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2006
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2007
https://www.transfermarkt.com/kawasaki-frontale/startseite/verein/9598?saison_id=2008
https://www.transfermarkt.com/kawasaki-frontale/starts

'Data inserted successfully'

In [10]:
matches = []
for year in range(startyear, endyear+1):
     
    try : 

        matches_url = f'https://www.transfermarkt.com/{club}/spielplan/verein/{club_id}/plus/1?saison_id='
        matches_url = matches_url + str(year)
        print(matches_url)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu") 
        user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        chrome_options.add_argument(f"user-agent={user_agent}")

        #driver = webdriver.Chrome(options=chrome_options) 
        driver = webdriver.Chrome() 
        driver.get(matches_url)

        driver.implicitly_wait(15) 
        page_source = driver.page_source
        driver.quit()
        #time.sleep(10)
        df = pd.read_html(page_source)
        soup = BeautifulSoup(page_source, 'html.parser')
        
        table_header_div = soup.find_all('h2', class_='content-box-headline')

        # if year >= 2008: 
        #     table_header_div = soup.find_all('div', class_='table-header')
        # else:
        #     table_header_div = soup.find_all('h2', class_='content-box-headline')
        header_vals = []

        for div in table_header_div:
            
            img_tag = div.find('img')
            if img_tag:
                header_val = img_tag.get('alt')
                header_vals.append(header_val)

        tables_div = soup.find_all('div', class_='responsive-table')

        dataframes = []
        for div in tables_div:
            df = pd.read_html(str(div))[0]
            dataframes.append(df)

        for i, df in enumerate(dataframes):
            df['Comp'] = header_vals[i] 
            df['Season'] = str(year) + '-' + str(year+1)

        matches_df = pd.concat(dataframes)
        matches.append(matches_df)

    except Exception as e:
        print(f"Could not fetch match logs for : {matches_url}")

match_stats = pd.concat(matches)
match_stats['Home team.1'] = match_stats['Home team.1'].str.replace(r'\(.*\)', '', regex=True)
match_stats['Away team.1'] = match_stats['Away team.1'].str.replace(r'\(.*\)', '', regex=True)
match_stats = match_stats.rename(columns={'Home team.1' : 'HomeTeam', 'Away team.1': 'AwayTeam'})
insert_into_db(match_stats, f'transfermarket_matches_{club}')

https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1992
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1993
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1994
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1995
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1996
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1997
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1998
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=1999
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=2000
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=2001
https://www.transfermarkt.com/ajax-amsterdam/spielplan/verein/610/plus/1?saison_id=2002
https://www.transfermarkt.com/aj

'Data inserted successfully'

In [12]:
print(f'transfermarket_matches_{club}')

transfermarket_matches_kawasaki-frontale
